In [1]:
import matplotlib.pyplot as plt
import pandas as pd
# Bring in evaluator
from sklearn import metrics
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.svm import SVC
# Flatten Data
from sklearn.preprocessing import StandardScaler, RobustScaler

%matplotlib inline

In [2]:
train_df = pd.read_csv('../dump/train_modified.csv')
test_df = pd.read_csv('../dump/test_modified.csv')
target='acc_now_delinq'
IDcol = 'member_id'
predictors = [x for x in train_df.columns if x not in [target,IDcol]]
train_df[target].value_counts()

0.0    697707
1.0      3258
Name: acc_now_delinq, dtype: int64

In [3]:
# Split Train/Test data
X_train, X_test, y_train, y_test = train_test_split (train_df[predictors], train_df[target], test_size=0.2, random_state=44)

In [4]:
# Weighted prediction feature
y_0 = len(y_train[y_train == 0])/len(y_train)
y_1 = 1 - y_0

In [5]:
#std_scaler = StandardScaler()
rob_scaler = RobustScaler()

#X_train_S = std_scaler.fit_transform(X_train)
#X_test_S = std_scaler.transform(X_test)

# Use robust scaler to reduce outliers
X_train_R = rob_scaler.fit_transform(X_train)
X_test_R = rob_scaler.transform(X_test)

In [ ]:
svm_clf = SVC(class_weight={0:y_1, 1:y_0})
svm_clf.fit(X_train_R, y_train)

svm_predictions = svm_clf.predict(X_test_R) # Save prediction


#print(svm_clf.score(X_test_R, y_test))
scores = cross_val_score(svm_clf, X_test_R, y_test, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std() * 2))

print(mt.classification_report(y_test, svm_predictions))
print(mt.confusion_matrix(y_test, svm_predictions))